In [ ]:
import os  # OS operations like file handling
import random  # Random number generation
import pandas as pd  # Data manipulation 
from PIL import Image  # Image processing
import cv2  # Computer vision tasks
from IPython.display import Video  # Display videos in Jupyter notebooks
import numpy as np  # Numerical computations
import matplotlib.pyplot as plt  # Plotting graphs and visualizations
import seaborn as sns  # Statistical data visualizations
sns.set_theme(style='darkgrid')
import pathlib  # File system paths handling
import glob  # File pattern matching (e.g., search for files)
from tqdm.notebook import trange, tqdm  # Progress bars in notebooks
import warnings  # Manage warning messages
warnings.filterwarnings('ignore')
from ultralytics import YOLO 

In [ ]:
# Final_model = YOLO('yolov11n.pt')
Final_model = YOLO("./runs/detect/train2/weights/last.pt")
# Training The Final Model
Result_Final_model = Final_model.train(
    data="/Users/limjh/Projects/BlackjackAI/data/data.yaml",
    device="mps",
    resume=True,
    save_period=1,
    epochs=60,
    batch=-1,
    optimizer="auto",
)

In [ ]:
Valid_model = YOLO("./runs/detect/train2/weights/best.pt")

In [ ]:
dataset_path = "./data"
test_images_path = os.path.join(dataset_path, "test", "images")

# List of all jpg images in the directory
image_files = [file for file in os.listdir(test_images_path) if file.endswith(".jpg")]

In [ ]:
if len(image_files) > 0:
    # Select 9 images at equal intervals
    num_images = len(image_files)
    step_size = max(1, num_images // 9)  # Ensure the interval is at least 1
    selected_images = [image_files[i] for i in range(0, num_images, step_size)]

    # Prepare subplots
    fig, axes = plt.subplots(3, 3, figsize=(20, 21))
    fig.suptitle("Validation Set Inferences", fontsize=24)

    for i, ax in enumerate(axes.flatten()):
        if i < len(selected_images):
            image_path = os.path.join(test_images_path, selected_images[i])
            # Load image
            image = cv2.imread(image_path)

            # Check if the image is loaded correctly
            if image is not None:
                # Resize image
                resized_image = cv2.resize(image, (640, 640))  # Direct resize
                normalized_image = resized_image / 255.0  # Normalize
                # Convert the normalized image to uint8 data type
                normalized_image_uint8 = (normalized_image * 255).astype(np.uint8)

                # Predict with the model
                results = Valid_model.predict(
                    source=normalized_image_uint8, imgsz=640, conf=0.5
                )

                # Plot image with labels
                annotated_image = results[0].plot(line_width=1)
                annotated_image_rgb = cv2.cvtColor(annotated_image, cv2.COLOR_BGR2RGB)
                ax.imshow(annotated_image_rgb)
            else:
                print(f"Failed to load image {image_path}")
        ax.axis("off")

    plt.tight_layout()
    plt.show()